In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
import missingno as msno

from sklearn.linear_model import LinearRegression

In [2]:
tr = pd.read_csv("datasets/train.csv")

***Data Cleaning***

In [10]:
thresh = int(tr.shape[0]*.9)

In [4]:
tr.dropna(axis = 1, thresh = thresh, inplace = True)

In [7]:
tr.dropna(axis = 0, inplace = True)